In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.svm import SVC
from joblib import dump,load
import os
from sklearn.metrics import accuracy_score, classification_report
def fetch_data(ticker):
    stock = yf.Ticker(ticker)
    data = stock.history(period="max")  # Fetch 5 years of data
    data.reset_index(inplace=True)
    return data


In [2]:
stock_list=pd.read_csv('nasdaq-listed.csv')

In [3]:

def preprocess_data(data):
    # Feature engineering: Create relevant columns
    data['open-close'] = data['Open'] - data['Close']
    data['high-low'] = data['High'] - data['Low']
    data['price-change'] = data['Close'].pct_change()
    data['is_quarter_end'] = np.where((data['Date'].dt.month % 3 == 0) & (data['Date'].dt.day > 23), 1, 0)
    data['SMA_10'] = data['Close'].rolling(window=10).mean()
    data['SMA_50'] = data['Close'].rolling(window=50).mean()
    data['SMA_200'] = data['Close'].rolling(window=200).mean()
    data['EMA_10'] = data['Close'].ewm(span=10, adjust=False).mean()
    data['EMA_50'] = data['Close'].ewm(span=50, adjust=False).mean()
    data['EMA_200'] = data['Close'].ewm(span=200, adjust=False).mean()
    # Drop unnecessary columns
    data = data.drop(['Dividends', 'Stock Splits'], axis=1, errors='ignore')
    data.dropna(inplace=True)  # Handle missing values
    
    return data




In [4]:
accuracy_list=pd.read_csv("Accuracy_Data_NASDAQ.csv")

print(accuracy_list)

      STOCK   Accuracy Symbol
0       NaN  55.911330   AACG
1       NaN  54.248366   AADI
2       NaN  53.197674   AADR
3       NaN  49.354839    AAL
4       NaN  67.598017   AAME
...     ...        ...    ...
3759    NaN  62.280306   USEG
3760    NaN  46.875000   USFI
3761    NaN  58.695652   USGO
3762    NaN  53.810624   USIG
3763    NaN  70.919176   USIO

[3764 rows x 3 columns]


In [ ]:
for index, row in stock_list.iloc[4428:].iterrows():
    stock=row['Stocks']
    data=fetch_data(stock)
    if (not data.empty) & (len(data)>11):
        data=preprocess_data(data)
        model_dir=f"./models/{stock}/"
        if not os.path.exists(model_dir):
                    os.mkdir(model_dir)
        accuracy_list.to_csv("Accuracy_Data_NASDAQ.csv",index=False)
        # Define features (X) and target (y)
        features = data[['open-close', 'high-low', 'Volume', 'is_quarter_end','SMA_10','SMA_50',"SMA_200",'EMA_10','EMA_50',"EMA_200"]]
        target = np.where(data['Close'].shift(-1) > data['Close'], 1, 0)  # 1 foSr price increase, 0 otherwise
        if features.shape[0] > 0 and target.shape[0] > 0:
   
            X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
            
            # Standardize features
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)
            # Initialize and train the model
            model = SVC(kernel='poly', probability=True, random_state=42)  
            model.fit(X_train, y_train)
            
            # Save the scaler and model for reuse
            
            dump(scaler, f"{model_dir}{stock}_scaler.joblib")
            dump(model, f"{model_dir}{stock}_predictor.joblib")
            y_pred = model.predict(X_test)
            
            # Evaluate performance
            print(stock," ",index)
            print("Accuracy:", accuracy_score(y_test, y_pred))
            print("Classification Report:\n", classification_report(y_test, y_pred))
            stock_accuracy=pd.DataFrame({"Symbol":[row['Stocks']],"Accuracy":[accuracy_score(y_test,y_pred)*100]})
            accuracy_list=pd.concat([accuracy_list,stock_accuracy])
            print("Accuracy List Length :",len(accuracy_list))
        

USTB   4429
Accuracy: 0.5527950310559007
Classification Report:
               precision    recall  f1-score   support

           0       0.50      0.02      0.04       144
           1       0.55      0.98      0.71       178

    accuracy                           0.55       322
   macro avg       0.53      0.50      0.37       322
weighted avg       0.53      0.55      0.41       322

Accuracy List Length : 3768
USVM   4430
Accuracy: 0.48757763975155277
Classification Report:
               precision    recall  f1-score   support

           0       0.18      0.01      0.02       158
           1       0.50      0.95      0.65       164

    accuracy                           0.49       322
   macro avg       0.34      0.48      0.34       322
weighted avg       0.34      0.49      0.34       322

Accuracy List Length : 3769
USVN   4431
Accuracy: 0.4
Classification Report:
               precision    recall  f1-score   support

           0       0.50      0.03      0.06        30


VACHW: Period 'max' is invalid, must be one of ['1d', '5d']


VALN   4447
Accuracy: 0.5655172413793104
Classification Report:
               precision    recall  f1-score   support

           0       0.57      0.93      0.70        81
           1       0.54      0.11      0.18        64

    accuracy                           0.57       145
   macro avg       0.55      0.52      0.44       145
weighted avg       0.56      0.57      0.47       145

Accuracy List Length : 3783
VALU   4448
Accuracy: 0.5975728155339806
Classification Report:
               precision    recall  f1-score   support

           0       0.60      0.98      0.75      1232
           1       0.49      0.02      0.04       828

    accuracy                           0.60      2060
   macro avg       0.54      0.50      0.39      2060
weighted avg       0.55      0.60      0.46      2060

Accuracy List Length : 3784
VANI   4449
Accuracy: 0.5808510638297872
Classification Report:
               precision    recall  f1-score   support

           0       0.58      1.00      0

VCICW: Period 'max' is invalid, must be one of ['1d', '5d']


VCIG   4458
Accuracy: 0.574468085106383
Classification Report:
               precision    recall  f1-score   support

           0       0.57      1.00      0.73        27
           1       0.00      0.00      0.00        20

    accuracy                           0.57        47
   macro avg       0.29      0.50      0.36        47
weighted avg       0.33      0.57      0.42        47

Accuracy List Length : 3791


...

C:\Users\TANUJ\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\TANUJ\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\TANUJ\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


VSMV   4556
Accuracy: 0.5457227138643068
Classification Report:
               precision    recall  f1-score   support

           0       0.33      0.07      0.11       144
           1       0.57      0.90      0.69       195

    accuracy                           0.55       339
   macro avg       0.45      0.48      0.40       339
weighted avg       0.47      0.55      0.45       339

Accuracy List Length : 3883
VSSYW   4557
Accuracy: 0.725
Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.95      0.84       121
           1       0.14      0.03      0.04        39

    accuracy                           0.72       160
   macro avg       0.45      0.49      0.44       160
weighted avg       0.60      0.72      0.65       160

Accuracy List Length : 3884
VSTA   4558
Accuracy: 0.6120218579234973
Classification Report:
               precision    recall  f1-score   support

           0       0.61      0.99      0.76       11

VSTEW: Period 'max' is invalid, must be one of ['1d', '5d']


VSTM   4561
Accuracy: 0.5482815057283142
Classification Report:
               precision    recall  f1-score   support

           0       0.56      0.97      0.71       342
           1       0.18      0.01      0.01       269

    accuracy                           0.55       611
   macro avg       0.37      0.49      0.36       611
weighted avg       0.39      0.55      0.40       611

Accuracy List Length : 3887
VTC   4562
Accuracy: 0.5625
Classification Report:
               precision    recall  f1-score   support

           0       0.33      0.07      0.11       131
           1       0.58      0.90      0.71       189

    accuracy                           0.56       320
   macro avg       0.46      0.49      0.41       320
weighted avg       0.48      0.56      0.47       320

Accuracy List Length : 3888
VTGN   4563
Accuracy: 0.6464174454828661
Classification Report:
               precision    recall  f1-score   support

           0       0.66      0.96      0.78       419

C:\Users\TANUJ\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\TANUJ\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\TANUJ\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
accuracy_list.to_csv("Accuracy_Data_NASDAQ.csv",index=False)